In [5]:
import sys
sys.path.append('../')

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from dataset.omsignal_dataset import OMSignalDataset
#from dataset.unlabeled_dataset import UnlabeledOMSignalDataset
from torchvision.transforms import Compose

from dataset.preprocessor import Preprocessor
from dataset.segmenter import PairedHeartBeatSegmenter,HeartBeatSegmenter,RandomPairedHeartBeatSegmenter
from dataset.to_float_tensor_transform import ToFloatTensorTransform
from dataset.flip_transform import FlipTransform
from dataset.shift_transform import ShiftTransform
from dataset.gaussian_noise_transform import GaussianNoiseTransform
from dataset.normalize_transform import Normalize
from dataset.encode_transform import Encode

import matplotlib.pyplot as plt

from models.multi_task_model import MultiTaskNet,MultiTaskNetRegression
from models.multi_task_trainer import MultiTaskTrainer

from utils.dataset_utils import generate_transformed_dataset
from utils.ids_conversion import convert_back_ids,convert_ids

from models.multi_task_predictions import multi_task_predict

from utils.file_utils import read_data
from scipy.stats import kendalltau, mode
from sklearn.metrics import accuracy_score

import pandas as pd
import time

from utils.dataset_utils import generate_random_dataset

In [6]:
TRAIN_UNLABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_UnlabeledData.dat"
TRAIN_LABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_TrainLabeledData.dat"
VALID_LABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_ValidationLabeledData.dat"

SEG_TRAIN_UNLABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_SegmentedData_Backup_2.dat"
SEG_TRAIN_LABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_Segmented_TrainLabeledData.dat"
SEG_VALID_LABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_Segmented_ValidationLabeledData.dat"

SAVE_PATH = "./results/"

In [7]:

# origin_dataset_path = TRAIN_LABELED_PATH
# destination_dataset_path = SEG_TRAIN_LABELED 
# transforms = Compose([Preprocessor(), PairedHeartBeatSegmenter(230)])
# generate_random_dataset(True,
#                          origin_dataset_path,
#                          destination_dataset_path,
#                          transforms,
#                          nsegments = 8,
#                          number_of_average = 100,
#                          Encode(encoder))

# origin_dataset_path = VALID_LABELED_PATH
# destination_dataset_path = SEG_VALID_LABELED 
# transforms = Compose([Preprocessor(), PairedHeartBeatSegmenter(230)])
# generate_random_dataset(True,
#                          origin_dataset_path,
#                          destination_dataset_path,
#                          transforms,
#                          nsegments = 8,
#                          number_of_average =100,
#                          Encode(encoder))

In [8]:
'''
Prepare dataset and dataloader
'''

transforms_train = Compose([ToFloatTensorTransform()])

transforms_valid =  Compose([ToFloatTensorTransform()])

# transforms_valid = Compose([Preprocessor(),
#                             PairedHeartBeatSegmenter(take_average=True),
#                             ToFloatTensorTransform()])

train_labeled_data = OMSignalDataset(True,True,SEG_TRAIN_LABELED,transforms_train)
valid_labeled_data = OMSignalDataset(True,True,SEG_VALID_LABELED ,transforms_valid)
# valid_labeled_data = OMSignalDataset(False,True,VALID_LABELED_PATH,transforms_valid)

trainloader = torch.utils.data.DataLoader(train_labeled_data, batch_size=16,
                                          shuffle=True)

validloader = torch.utils.data.DataLoader(valid_labeled_data , batch_size=16,
                                         shuffle=False)

In [9]:
'''
train model

'''
model = MultiTaskNet(230, 1, 35, 30)
Trainer = MultiTaskTrainer(model, 
                           trainloader,
                           validloader,
                           SAVE_PATH, 
                           learning_rate=0.0001, 
                           batch_size=16, 
                           weight_decay=0.01,
                           log=True,
                           seed=111)

train_losses,valid_losses= Trainer.train(50)

Start training.
Epoch : 0
Start validation.
Epoch train total loss : 12.384326934814453
Epoch valid total loss : 10.964822769165039
Epoch train ID loss : 3.326066493988037
Epoch valid ID loss : 3.4082143306732178
Epoch train PR loss : 13.624890327453613
Epoch valid PR loss : 12.674955368041992
Epoch train RT loss : 31.02537727355957
Epoch valid RT loss : 25.29583168029785
Epoch train RR loss : 3.9671103954315186
Epoch valid RR loss : 3.2204787731170654
Accuracy on train data : 21.29019372877971 %
Accuracy on valid data : 7.025380710659898 %
Epoch : 1
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 4.521618366241455
Epoch valid total loss : 4.256016731262207
Epoch train ID loss : 2.826747417449951
Epoch valid ID loss : 3.5511715412139893
Epoch train PR loss : 3.386054515838623
Epoch valid PR loss : 2.3031435012817383
Epoch train RT loss : 5.675525188446045
Epoch valid RT loss : 2.3013386726379395
Epoch train RR loss : 2.239522933959961
Epoch valid 

Epoch valid ID loss : 3.8344545364379883
Epoch train PR loss : 0.6639919281005859
Epoch valid PR loss : 1.2865761518478394
Epoch train RT loss : 0.9090127944946289
Epoch valid RT loss : 0.9488791823387146
Epoch train RR loss : 1.227852702140808
Epoch valid RR loss : 2.6913516521453857
Accuracy on train data : 98.58198522069104 %
Accuracy on valid data : 50.74111675126903 %
Epoch : 15
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 0.6431356072425842
Epoch valid total loss : 4.280333518981934
Epoch train ID loss : 0.1169118583202362
Epoch valid ID loss : 3.9906060695648193
Epoch train PR loss : 0.6714797616004944
Epoch valid PR loss : 1.3522937297821045
Epoch train RT loss : 0.8717461228370667
Epoch valid RT loss : 1.6191308498382568
Epoch train RR loss : 1.2048053741455078
Epoch valid RR loss : 2.4678168296813965
Accuracy on train data : 97.78310365488316 %
Accuracy on valid data : 51.817258883248726 %
Epoch : 16
Start validation.
Epoch train tota

Epoch valid total loss : 3.523263931274414
Epoch train ID loss : 0.06869513541460037
Epoch valid ID loss : 3.2417819499969482
Epoch train PR loss : 0.509942889213562
Epoch valid PR loss : 1.1796112060546875
Epoch train RT loss : 0.8088379502296448
Epoch valid RT loss : 0.9901785850524902
Epoch train RR loss : 0.9558992981910706
Epoch valid RR loss : 2.4793992042541504
Accuracy on train data : 99.60055921709606 %
Accuracy on valid data : 55.898477157360404 %
Epoch : 30
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 0.47259822487831116
Epoch valid total loss : 3.5770556926727295
Epoch train ID loss : 0.06325321644544601
Epoch valid ID loss : 3.3276138305664062
Epoch train PR loss : 0.4737570285797119
Epoch valid PR loss : 1.1393892765045166
Epoch train RT loss : 0.6710942983627319
Epoch valid RT loss : 0.7772575616836548
Epoch train RR loss : 0.9651265740394592
Epoch valid RR loss : 2.658175468444824
Accuracy on train data : 99.4008388256441 %
Accu

Saving model multi_task_net to : ./results/
Epoch train total loss : 0.43615445494651794
Epoch valid total loss : 3.5143706798553467
Epoch train ID loss : 0.05119461938738823
Epoch valid ID loss : 3.2204318046569824
Epoch train PR loss : 0.43645739555358887
Epoch valid PR loss : 1.1565898656845093
Epoch train RT loss : 0.7026939988136292
Epoch valid RT loss : 1.0928764343261719
Epoch train RR loss : 0.8368427753448486
Epoch valid RR loss : 2.440659999847412
Accuracy on train data : 99.80027960854804 %
Accuracy on valid data : 55.065989847715734 %
Epoch : 45
Start validation.
Saving model multi_task_net to : ./results/
Epoch train total loss : 0.42789262533187866
Epoch valid total loss : 3.556867837905884
Epoch train ID loss : 0.04933701828122139
Epoch valid ID loss : 3.269170045852661
Epoch train PR loss : 0.4459804594516754
Epoch valid PR loss : 1.2165544033050537
Epoch train RT loss : 0.6827221512794495
Epoch valid RT loss : 0.987117350101471
Epoch train RR loss : 0.8134128451347351


<Figure size 432x288 with 0 Axes>

In [10]:
'''
predict

'''
model = MultiTaskNet(230, 1, 35, 30)
model.load_state_dict(torch.load('./results/multi_task_net.pt'))

test_transforms = Compose([Preprocessor(),
                           PairedHeartBeatSegmenter(230)
                           ])

test_data = OMSignalDataset(False, True, VALID_LABELED_PATH, test_transforms)

y_pred,true_labels = multi_task_predict(model,test_data)

In [11]:
'''
print Metrics

'''

converted_ids = convert_ids(true_labels[:,3])

print("ID: " + str(accuracy_score(converted_ids,y_pred[:,3])))
print("PR: " + str(kendalltau(true_labels[:,0],y_pred[:,0])[0]))
print("RT: " + str(kendalltau(true_labels[:,1],y_pred[:,1])[0]))
print("RR: " + str(kendalltau(true_labels[:,2],y_pred[:,2])[0]))

ID: 0.64375
PR: 0.7424123381624358
RT: 0.8493473921659216
RR: 0.3755650775337741
